txt file, в котором в первой строке написано название модели
(его можно взять из конфиг файла)
во второй строке написан acc
в третьей строке написан loss
во четвертой строке написан val acc
в пятой строке написан val loss
сохранить png картинку с графиком (сделать все на один график и лосс и вал лосс и акк и вал акк)
немесил в пакетике

In [19]:
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from os.path import join
import efficientnet.tfkeras as efn
import pandas as pd

In [20]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [21]:
model_name = 'PSU_model'
model_path = join('output',model_name,'model.json')
weights_path = join('output',model_name,'model_weights.h5')
history_path = join('output',model_name,'history')
csv_path = 'data/test_pathology.csv'

In [22]:
def prepare_model(model_path,weights_path,history):
    with open(model_path,'r') as f:
        json_config = f.read()
        model = tf.keras.models.model_from_json(json_config)
        model.load_weights(weights_path)
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(history["lr"][len(history["lr"])-1]),
                      metrics=['accuracy'])
        return model

In [23]:
def load_history(history_path):
    with open(history_path,'rb') as f:
        return pickle.load(f)

In [24]:
def get_test_data(path):
    df = pd.read_csv(path)
    paths = df.pop('path')
    paths = [str(elem) for elem in paths]
    y = df.to_numpy().astype('float32')
    return np.array(paths),y

In [31]:
def get_prediction(history):
    
    model = prepare_model(model_path,weights_path,history)
    
    pics_names,lbls = get_test_data(csv_path)
    pics = []
    
    for pic in pics_names:
        pic_ = np.load(pic).astype('float32')/255
        pics.append(pic_)
    
    pics = np.asarray(pics)
    
    result = model.evaluate(pics,lbls)
    return result[0], result[1]

In [33]:
with open(join('output',model_name,'report.txt'),'w') as f:
        #с истории выколупываем 
        f.write('model name: '+ model_name+'\n')
        history = load_history(history_path)
        #
        test_loss, test_acc = get_prediction(history)
        #test_loss,test_acc = 0.5,0.3
        #сохраняется картинОЧКА
        figure, _ = plt.subplots()
        plt.rcParams["figure.figsize"] = (8,8)
        plt.plot(history["accuracy"])
        plt.plot(history['val_accuracy'])
        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        plt.title("Accuracy and Loss")
        plt.ylabel("Value")
        plt.xlabel("Epoch")
        plt.legend(["Accuracy","Val. Accuracy","Loss","Validation Loss"])
        figure.savefig(join('output',model_name,'plots.png'))
        #
        f.write('train accuracy: '+str(history["accuracy"][len(history["accuracy"])-1])+'\n')
        f.write('validation accuracy: '+str(history["val_accuracy"][len(history["val_accuracy"])-1])+'\n')
        f.write('test accuracy: '+str(test_acc)+'\n')
        f.write('train loss: '+str(history["loss"][len(history["loss"])-1])+'\n')
        f.write('validation loss: '+str(history["val_loss"][len(history["val_loss"])-1])+'\n')
        f.write('test loss: '+str(test_loss)+'\n')

  7/457 [..............................] - ETA: 4:25 - loss: 4.8057 - accuracy: 0.0000e+00

KeyboardInterrupt: 